## Importar librerías

In [1]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime as dt
import datetime
from datetime import date
import string
import unicodedata
import os

## Utilidades

In [2]:
def rreplace(s, old, new, n_occurrence):
    # Replace the last n_occurrence (n_occurrence = 1 is the last occurrence) of an expression in a string s
    li = s.rsplit(old, n_occurrence)
    return new.join(li)

def str_format(stringg, sep = '_'):
    
    #Elimina simbolos de puntuación, u otros simbolos de un string
    #Usar sep = "_" para nombre de columnas, usar sep = ' ' para contenido
    
    punctuation = list(string.punctuation)          
    symbols = ['^','°', 'ª', os.linesep, '\n']                     
    new_string = unicodedata.normalize('NFKD', str(stringg)).encode('ascii', errors='ignore').decode('utf-8') # Elimina tildes 

    if sep == '_':
        for i in range(len(punctuation)):
            new_string = new_string.replace(punctuation[i], sep)   # Elimina simbolos de puntuación
    else:
        new_string = new_string.replace(';','.')
    
    for i in range(len(symbols)):
        new_string = new_string.replace(symbols[i], '')           # Elimina otros simbolos
    
    if sep == '_':
        new_string = new_string.replace(' ','_').lower()         # Elimina espacios 
    
    while new_string[-1:] == sep:
        new_string = rreplace(new_string,sep,'',1)           # Elimina los "espacios" (_) al final del string
    
    return new_string


def read_txt(path, header_int, sep = '|', encoding = 'latin1'):
# Lee los archivos txt del path (correspondientes a la fecha de descarga), y crea diccionario con respectivos dataframes
    all_files = pd.Series(glob.glob(path + "/*.txt"))
    fecha_str = date.today().strftime("%Y%m")
    files = all_files[all_files.astype(str).str.contains(fecha_str)].tolist()
    d = {}
    
    for i in range(0,len(files)):
        file_path = files[i]
        index_from = file_path.rfind("\\") + 1
        file_name = str_format(file_path[index_from:-4])
        
        if file_name[:8].isdigit():
            file_name = file_name[9:]
        
        d[str(file_name)] = pd.read_csv(file_path,header=header_int, sep=sep, encoding=encoding, dtype=str)
    return d

def clean_columns(dict, del_from, col=0):
    for keys in dict:
        # Elimina la primera y última columna 
        dict[keys] = dict[keys].drop(columns=dict[keys].columns[0]).\
        drop(columns = dict[keys].columns[-1]).dropna(how = 'all').reset_index(drop = True)
        del_from_index = dict[keys].index[dict[keys][dict[keys].columns[col]].str.contains(del_from)].tolist()[0]
        
        #Elimina filas al final de txt que presentan el resumen del libro de compras/ventas
        dict[keys] = dict[keys].iloc[0:del_from_index,:]
    return dict

def save_as_csv(dict, path, suffixes='', sep=';', encoding = 'latin1', index = False):
    
    for keys in dict:
        dict[keys].to_csv(path + '\\' + dt.now().strftime('%Y%m%d') + str(keys) + suffixes + '.csv', sep=sep, index=index)
        
def read_xlsx(path, header_int, sheet_name_str=0):
# Lee los archivos xlsx del path (correspondientes a la fecha de descarga), y crea diccionario con respectivos dataframes
    all_files = pd.Series(glob.glob(path + "/*.xlsx"))
    fecha_str = date.today().strftime("%Y%m")
    files = all_files[all_files.astype(str).str.contains(fecha_str)].tolist()
    d = {}
    
    for i in range(0,len(files)):
        file_path = files[i]
        index_from = file_path.rfind("\\") + 1
        file_name = str_format(file_path[index_from:-4])
        
        if file_name[:8].isdigit():
            file_name = file_name[9:]
        
        d[str(file_name)] = pd.read_excel(file_path, header=header_int, sheet_name=sheet_name_str, dtype=str)
    return d

def col_names_format(dict):
    
    for keys in dict:
        df_col = dict[keys].columns
        col_list = []
        for col in df_col:
            col_list.append(str_format(col))
        dict[keys].columns = col_list
    
    return dict

def schema_cols(dict, col_names, empty_col_value = np.nan):
    for keys in dict:
        for col in col_names:
            if col not in dict[keys].columns:
                dict[keys][col] = empty_col_value
        dict[keys] = dict[keys][col_names]
    return dict

def replace_unnamed_cols(dict):
    columns = []
    for keys in dict:
        for col in dict[keys].columns:
            if 'Unnamed' in str(col):
                col = ''
            columns.append(col)
    dict[keys].columns = columns
    return dict

### Data paths

In [3]:
input_path = r'C:/projects/data_lake/data/input/current/nomina_macroprocesos'
output_path = r'C:/projects/data_lake/data/output/current/nomina_macroprocesos'

## Ejecutar limpieza
### Leer archivos y guardar en diccionario dict

In [4]:
dict = read_xlsx(input_path,1,"Proceso - MRC (SP)")

### Cambiar formato de nombre de columnas

In [5]:
dict = col_names_format(dict)

In [6]:
for key in dict:
    print(key,dict[key].columns)

20190102_mapa_de_proceso_achs Index(['id', 'n1', 'categoria', 'n2', 'macroproceso', 'n3', 'proceso',
       'prioridad_2017', 'prioridad_2018', 'linea_de_negocio',
       'gerencia_responsable', 'gerencia___subgerencia_responsable',
       'fecha_de_ultima_actualizacion', 'ano', 'numero_de_actualizacion',
       'proyecto_de_mejora', 'mrcvigente', 'nombre_mrc', 'act_2017',
       'fecha_ultima_actualizacion_mrc', 'pendiente_actualizacion',
       'fecha_proxima_actualizacion', 'grupo', '1_era_reunion', 'estado',
       '2da_reunion', 'estado__1', 'observacion', '3era_reunion', 'estado__2',
       'observacion__1', 'estado', 'estado_mrc', '1era_reunion_mrc',
       'estado_mrc_1', '2da_reunion_mrc', 'observacion', 'dueno_macroproceso',
       'dueno_proceso', 'lider_proceso', 'gestor_de_proceso', 'cod_per_dm',
       'cod_per_dp', 'cod_per_lp', 'cod_per_gp', 'vigente_dm', 'vigente_dp',
       'vigente_lp', 'vigente_gp', 'validacion_uo_dp', 'validacion_uo_lp',
       'validacion_uo_gp', 

### Establecer columnas que serán parte del esquema

In [7]:
col_names = ['id', 'categoria', 'macroproceso', 'proceso', 'prioridad2018', 'prioridad_2017', 'prioridad_2018',
       'fecha_de_ultima_actualizacion', 'mrc_vigente','mrcvigente', 'nombre_mrc',
       'fecha_ultima_actualizacion_mrc', 'fecha_de_actualizacionpco',
       'dueno_macroproceso', 'dueno_proceso', 'lider_proceso',
       'gestor_de_proceso']

dict = schema_cols(dict, col_names)

### Rellenar valores nulos con ""

In [8]:
for keys in dict:
    dict[keys].fillna('', inplace = True)

### Cambiar formato de datos

In [9]:
for keys in dict:
    dict[keys] = dict[keys].applymap(lambda x: str_format(x, sep=' '))

### Guardar en archivos csv

In [11]:
save_as_csv(dict, output_path)